In [ ]:
!gdown --id 0B7XkCwpI5KDYNlNUTTlSS21pQmM

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=0B7XkCwpI5KDYNlNUTTlSS21pQmM
From (redirected): https://drive.google.com/uc?id=0B7XkCwpI5KDYNlNUTTlSS21pQmM&confirm=t&uuid=3f2b8084-8f6e-48d1-94cb-929185e508fc
To: /content/GoogleNews-vectors-negative300.bin.gz
100% 1.65G/1.65G [00:20<00:00, 81.5MB/s]


In [ ]:
!mkdir plagChecking



mkdir: cannot create directory ‘plagChecking’: File exists


In [ ]:
import numpy as np
from gensim.models.keyedvectors import KeyedVectors

# use this command to download the file - !wget -P /root/input/ -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"
saved_vectors = 'GoogleNews-vectors-negative300.bin.gz'
EMBEDDING_FILE = 'GoogleNews-vectors-negative300.bin.gz'

#converting answers of the target student to a string
with open("plagChecking/answer_sheet1.txt", "r") as file:
    data1 = file.read().replace("\n", " ")

#converting answers of the other student to a list of strings
with open("plagChecking/answer_sheet2.txt", "r") as file:
    data2 = file.read().replace("\n", " ")

data2 = data2.split(" ")
answer_paper2 = []
for i in range(int(len(data2) / 5)):
    answer_paper2.append(data2[5 * i: (5 * i) + 5])

ans2 = []
for i in answer_paper2:
    empty = ""
    for j in i:
        empty += (j + " ")
    ans2.append(empty)


class DocSim:
    def __init__(self, w2v_model, stopwords=None):
        self.w2v_model = w2v_model
        self.stopwords = stopwords if stopwords is not None else []

    def vectorize(self, doc: str) -> np.ndarray:
        """
        Identify the vector values for each word in the given document
        :param doc:
        :return:
        """
        doc = doc.lower()
        words = [w for w in doc.split(" ") if w not in self.stopwords]
        word_vecs = []
        for word in words:
            try:
                vec = self.w2v_model[word]
                word_vecs.append(vec)
            except KeyError:
                # Ignore, if the word doesn't exist in the vocabulary
                pass

        # Assuming that document vector is the mean of all the word vectors
        # We could improvise here
        vector = np.mean(word_vecs, axis=0)
        return vector

    def _cosine_sim(self, vecA, vecB):
        """Find the cosine similarity distance between two vectors."""
        csim = np.dot(vecA, vecB) / (np.linalg.norm(vecA) * np.linalg.norm(vecB))
        if np.isnan(np.sum(csim)):
            return 0
        return csim

    def calculate_similarity(self, source_doc, target_docs=None, threshold=0):
        """Calculates & returns similarity scores between given source document & all
        the target documents."""
        if not target_docs:
            return []

        if isinstance(target_docs, str):
            target_docs = [target_docs]

        source_vec = self.vectorize(source_doc)
        results = []
        for doc in target_docs:
            target_vec = self.vectorize(doc)
            sim_score = self._cosine_sim(source_vec, target_vec)
            if sim_score > threshold:
                results.append({"score": sim_score, "doc": doc})
            # Sort results by score in desc order
            results.sort(key=lambda k: k["score"], reverse=True)

        return results


googlenews_model_path = EMBEDDING_FILE
stopwords_path = "stopwords_en.txt"

model = KeyedVectors.load_word2vec_format(googlenews_model_path, binary=True)
with open(stopwords_path, 'r') as fh:
    stopwords = fh.read().split(",")
ds = DocSim(model, stopwords=stopwords)

source_doc = data1
target_docs = ans2 # we can provide multiple student's answers to check between them
sim_scores = ds.calculate_similarity(source_doc, target_docs)
count = 0
for i in sim_scores:
    count += i[list(i.keys())[0]]
count /= len(sim_scores)
print(sim_scores)
print(count)

cheating_mark = 0.75
if count > cheating_mark:
    print("High level of Cheating Detected. Please check this answer!!!")


[{'score': 0.76359516, 'doc': 'simulation process, select input variables '}, {'score': 0.7419697, 'doc': 'variables of a simulation model '}, {'score': 0.7339794, 'doc': 'an appropriate simulation software. Step '}, {'score': 0.7259899, 'doc': 'decision variables to the simulation '}, {'score': 0.69131726, 'doc': 'to perform simulation analysis. Step '}, {'score': 0.68311447, 'doc': 'and create entities. Decision variables '}, {'score': 0.676798, 'doc': 'designing a simulation experiment. Step '}, {'score': 0.66773236, 'doc': 'Step 3. Perform simulation runs. '}, {'score': 0.66170055, 'doc': 'simulation modelling and analysis activity '}, {'score': 0.6597993, 'doc': 'to perform simulation analysis. Answer '}, {'score': 0.6569469, 'doc': 'a simulation experiment. Answer - '}, {'score': 0.6569469, 'doc': 'analysis. Answer - Simulation modelling '}, {'score': 0.65537655, 'doc': 'simulation run is appropriate. Select '}, {'score': 0.64947146, 'doc': 'A simulation experiment is a '}, {'sco

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
